In [67]:
import pandas as pd
import geopandas as gpd
import numpy as np
import random
from sklearn.neighbors import BallTree
from shapely.geometry import Point
from joblib import Parallel, delayed

In [68]:
# Load Residents Data
residents = pd.read_csv("Mo_pop_Sim.csv")
residents

,UR,long,lat
0,R,-92.506276,40.279203
1,R,-92.505132,40.276890
2,R,-92.505890,40.278242
3,R,-92.519842,40.327700
4,R,-92.524877,40.329511
...,...,...,...
6332487,U,-90.241253,38.636296
6332488,U,-90.240500,38.635959
6332489,U,-90.240954,38.636279
6332490,U,-90.240701,38.636424


In [69]:
residents_gdf = gpd.GeoDataFrame(residents, geometry=gpd.points_from_xy(residents.long, residents.lat))
residents_gdf


,UR,long,lat,geometry
0,R,-92.506276,40.279203,POINT (-92.50628 40.2792)
1,R,-92.505132,40.276890,POINT (-92.50513 40.27689)
2,R,-92.505890,40.278242,POINT (-92.50589 40.27824)
3,R,-92.519842,40.327700,POINT (-92.51984 40.3277)
4,R,-92.524877,40.329511,POINT (-92.52488 40.32951)
...,...,...,...,...
6332487,U,-90.241253,38.636296,POINT (-90.24125 38.6363)
6332488,U,-90.240500,38.635959,POINT (-90.2405 38.63596)
6332489,U,-90.240954,38.636279,POINT (-90.24095 38.63628)
6332490,U,-90.240701,38.636424,POINT (-90.2407 38.63642)


In [70]:
# Load FQHCs Data
fqhcs = gpd.read_file("MO_2018_Federally_Qualified_Health_Center_Locations.shp")
fqhcs

,OBJECTID,Group_Name,Facility,Address,City,County,State,Zip,Phone,Latitude,Longitude,Loc_Code,geometry
0,1,COMTREA,COMTREA Byrnes Mill Health Center,100 Osage Executive Circle,House Springs,Jefferson,MO,63051,6367893372,38.435946,-90.554678,MAP,POINT (-90.55472 38.43597)
1,2,Missouri Highlands Health Care,Viburnum Medical Clinic,18 Viburnum Center Road,Viburnum,Iron,MO,65566,5732445406,37.714620,-91.133983,MAP,POINT (-91.13403 37.71462)
2,3,Central Ozarks Medical Center,Central Ozarks Medical Center At The Lake,3870 Columbia Avenue,Osage Beach,Miller,MO,65065,5733027490,38.160258,-92.601463,MAP,POINT (-92.60144 38.16025)
3,4,Missouri Highlands Health Care,Missouri Highland Medical Clinic - Poplar Bluf...,"225 Physicians Park Drive, Suite 303",Poplar Bluff,Butler,MO,63901,5737856536,36.772568,-90.457206,MAP,POINT (-90.45724 36.77261)
4,5,Swope Health Services,Swope Health Hickman Mills,8800 Blue Ridge Boulevard,Kansas City,Jackson,MO,64138,8163213200,38.962882,-94.498847,MAP,POINT (-94.49886 38.9629)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,193,Your Community Health Center,Your Community Health Center - Health Department,"200 North Main, Suite G51",Rolla,Phelps,MO,65401,5734586950,37.945925,-91.773948,MAP,POINT (-91.77391 37.94594)
193,194,Jordan Valley Community Health Center,Jordan Valley Community Health Center - Medica...,"1443 N. Roberson, Suite 505",Springfield,Greene,MO,65802,4178511554,37.224321,-93.291591,MAP,POINT (-93.29166 37.2243)
194,195,Southern Missouri Community Health Center,Oregon County Community Health Center,"US Highway 63 North (RR 3, Box 3703)",Thayer,Oregon,MO,65791,4172642990,36.565336,-91.562618,MAP,POINT (-91.56258 36.56534)
195,196,CareSTL Health,Pope Avenue Health Center,4500 Pope Ave,St. Louis,St. Louis City,MO,63115,3143853990,38.677759,-90.230247,MAP,POINT (-90.23025 38.67777)


In [71]:
fqhcs = fqhcs[['Facility', 'Latitude', 'Longitude', 'geometry']]

fqhcs_gdf = gpd.GeoDataFrame(fqhcs, geometry=gpd.points_from_xy(fqhcs.Longitude, fqhcs.Latitude))
fqhcs_gdf


,Facility,Latitude,Longitude,geometry
0,COMTREA Byrnes Mill Health Center,38.435946,-90.554678,POINT (-90.55468 38.43595)
1,Viburnum Medical Clinic,37.714620,-91.133983,POINT (-91.13398 37.71462)
2,Central Ozarks Medical Center At The Lake,38.160258,-92.601463,POINT (-92.60146 38.16026)
3,Missouri Highland Medical Clinic - Poplar Bluf...,36.772568,-90.457206,POINT (-90.45721 36.77257)
4,Swope Health Hickman Mills,38.962882,-94.498847,POINT (-94.49885 38.96288)
...,...,...,...,...
192,Your Community Health Center - Health Department,37.945925,-91.773948,POINT (-91.77395 37.94592)
193,Jordan Valley Community Health Center - Medica...,37.224321,-93.291591,POINT (-93.29159 37.22432)
194,Oregon County Community Health Center,36.565336,-91.562618,POINT (-91.56262 36.56534)
195,Pope Avenue Health Center,38.677759,-90.230247,POINT (-90.23025 38.67776)


In [72]:
# Cluster Residents (Reduce Computation)
residents_gdf['lat_rounded'] = residents_gdf['lat'].round(2)  # Round to 2 decimal places (~1.2km grouping)
residents_gdf['lon_rounded'] = residents_gdf['long'].round(2)
clustered_residents = residents_gdf.groupby(['lat_rounded', 'lon_rounded']).size().reset_index(name='resident_count')

In [73]:
# Convert to BallTree-friendly format
residents_coords = np.radians(clustered_residents[['lat_rounded', 'lon_rounded']].values)
tree = BallTree(residents_coords, metric='haversine')

# Optimized Fitness Function (Parallel Processing)
def compute_fitness(fqhc_subset):
    selected_coords = np.radians(fqhc_subset[['Latitude', 'Longitude']].values)

# Define resident_count function inside compute_fitness()
    def resident_count(fqhc):
        distances, indices = tree.query(np.array([fqhc]), k=len(clustered_residents), return_distance=True)
        distances = distances * 3960  # Convert radians to miles
        return np.sum(clustered_residents.loc[indices[0], 'resident_count'][distances[0] <= 30])

# Parallel computation
    total_residents = sum(Parallel(n_jobs=-1)(delayed(resident_count)(fqhc) for fqhc in selected_coords))

    return total_residents

In [74]:
# Genetic Algorithm Components
def initialize_population(fqhcs_gdf, pop_size=10):
    return [fqhcs_gdf.sample(8) for _ in range(pop_size)]

def select_parents(population):
    population.sort(key=compute_fitness, reverse=True)
    return population[:5]  # Select top 5 for reproduction

def crossover(parent1, parent2):
    split = len(parent1) // 2
    child = pd.concat([parent1.iloc[:split], parent2.iloc[split:]]).drop_duplicates()
    if len(child) < 8:
        child = pd.concat([child, fqhcs_gdf.sample(8 - len(child))])
    return child.sample(8)

def mutate(child, fqhcs_gdf):
    child = child.copy()
    if random.random() < 0.2:
        idx = random.randint(0, len(child) - 1)
        child.iloc[idx] = fqhcs_gdf.sample(1).iloc[0]
    return child

In [75]:
# Run the Optimized Genetic Algorithm
population = initialize_population(fqhcs_gdf)
generations = 2

for _ in range(generations):
  parents = select_parents(population)
  offspring = [mutate(crossover(random.choice(parents), random.choice(parents)), fqhcs_gdf) for _ in range(5)]
  population = parents + offspring

best_solution = max(population, key=compute_fitness)

# Print Results
print("Optimal FQHCs for Mental Health Services:")
print(best_solution)

Optimal FQHCs for Mental Health Services:
                                              Facility   Latitude  Longitude  \
25            Affinia Healthcare (Program Mgmt Office)  38.650362 -90.193940   
16             Access Family Care - Cassville (Dental)  36.700193 -93.880715   
126                  Cabot Westside Medical and Dental  39.087119 -94.593263   
168                                       Macon Dental  39.739619 -92.465609   
84   Katy Trail Community Health - Prairie Hills Cl...  38.432267 -92.850884   
0                    COMTREA Byrnes Mill Health Center  38.435946 -90.554678   
33   Betty Jean Kerr People's Health Center, Inc. (...  38.654260 -90.285449   
87                           Big Springs Dental Clinic  37.241847 -90.969378   

                       geometry  
25   POINT (-90.19394 38.65036)  
16   POINT (-93.88072 36.70019)  
126  POINT (-94.59326 39.08712)  
168  POINT (-92.46561 39.73962)  
84   POINT (-92.85088 38.43227)  
0    POINT (-90.55468 38.43595)  

Interpretation:
Eight FQHC locations in Missouri were found to be the best for implementing specialized mental health services by the genetic algorithm. The fitness function prioritized locations with the highest number of residents within a 30-mile radius, ensuring services were introduced in high-density areas. The chosen FQHCs provide fair geographic coverage by spanning mid-sized towns (Macon, Jefferson County, Sedalia), rural areas (Cassville, Van Buren), and urban centers (St. Louis, Kansas City).
we utilized BallTree method to optimize spatial queries, resident clustering to minimize computational load, and parallel processing to enhance execution speed. The algorithm we used iteratively refined its selections using selection, crossover, and mutation, leading to progressively improved facility placement over multiple generations. This structured optimization ensured maximum community benefit while maintaining computational feasibility.
However, the model assumes uniform accessibility for all residents, overlooking factors such as transportation limitations, socioeconomic disparities, and varying demand levels for mental health services. Future refinements could involve incorporating real-world road networks for more accurate travel-time estimations, adjusting selection criteria to prioritize areas with higher service demand, and integrating socioeconomic data to better reflect community needs.
The genetic algorithm offered a scalable, data-driven method for healthcare facility placement optimization in spite of these drawbacks. It guarantees that mental health treatments are implemented where they are most required, significantly enhancing the wellbeing of Missouri's communities.